In [3]:
import regionmask
import geopandas
import xarray as xr
import numpy as np

In [7]:
# Open the base grid which to manipulate.
landsea=xr.open_dataset('/home/ucfaccb/miniconda3/envs/ncl-nco/lib/ncarg/data/cdf/landsea.nc')
landsea.LSMASK

ImportError: libtiff.so.5: cannot open shared object file: No such file or directory

In [ ]:
land_masks=regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat)
print(land_masks)
land_masks.plot()

In [ ]:
ocean_masks=regionmask.defined_regions.ar6.ocean.mask(landsea.lon, landsea.lat)
ocean_masks.plot()

In [ ]:
all_masks=regionmask.defined_regions.ar6.all.mask(landsea.lon, landsea.lat)
all_masks.plot()

In [ ]:
AR6_masks=landsea.LSMASK.copy()
AR6_masks.data=np.int8(regionmask.defined_regions.ar6.all.mask(landsea.lon, landsea.lat))
# Correct spelling of Indian Ocean
region_names=regionmask.defined_regions.ar6.all.names
region_names[55]="Equatorial.Indian-Ocean"
region_names[56]="S.Indian-Ocean"
AR6_masks=AR6_masks.assign_attrs(number=regionmask.defined_regions.ar6.all.numbers,abbrevs=regionmask.defined_regions.ar6.all.abbrevs,region_names=region_names)
AR6_masks=AR6_masks.assign_attrs(long_name="AR6 Regions",NUMBER=np.int8(regionmask.defined_regions.ar6.all.numbers),description="A combined mask to show the regions delineated by Iturbide et la (2020; Earth Syst. Sci. Data)")
AR6_masks

In [ ]:
# Add in some additional information to the file
number=regionmask.defined_regions.ar6.all.numbers
LAND=xr.DataArray(np.zeros(58,dtype=np.int8),dims="number")
LAND[0:46]=1
LAND=LAND.assign_attrs(long_name="Region defined over land")
OCEAN=xr.DataArray(np.zeros(58,dtype=np.int8),dims="number")
OCEAN[46:]=1
OCEAN=OCEAN.assign_attrs(long_name="Region defined over ocean")
BOTH=xr.DataArray(np.zeros(58,dtype=np.int8),dims="number")
BOTH=BOTH.assign_attrs(long_name="Region defined over both land and ocean")
MULTIPLY_DEF=xr.DataArray(np.zeros(58,dtype=np.int8),dims="number")
MULTIPLY_DEF[8]=1 # Caribbean
MULTIPLY_DEF[19]=1 # Mediterranean
MULTIPLY_DEF[38]=1 # S.E.Asia
MULTIPLY_DEF=MULTIPLY_DEF.assign_attrs(long_name="Is the region multiply defined (e.g. over both land and ocean)")
MULTIPLY_DEF


In [ ]:
AR6_masks_1x1 = xr.Dataset()
AR6_masks_1x1["AR6_masks"]=AR6_masks
AR6_masks_1x1["LAND"]=LAND
AR6_masks_1x1["OCEAN"]=OCEAN
AR6_masks_1x1["BOTH"]=BOTH
AR6_masks_1x1["MULTIPLY_DEF"]=MULTIPLY_DEF
AR6_masks_1x1


# Now add in some Monsoon Regions
Taken from Figure AV.1 of the Annex V of the IPCC: [https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_AnnexV.pdf]

This defines 7 regions of which five monsoon regions. SAfri and EqAmeri are not official monsoons (they are regions of seasonal precipitation instead), but I might as well compute them

In [ ]:
# Define the easy ones first...
AR6_monsoons=landsea.LSMASK.copy()
AR6_monsoons.data=np.full((180, 360), 9, dtype=np.int8) #set up a base array
monsoon_abbrevs=["NAmerM","EqAmer","SAmerM","WAfriM","SAfri","SAsiaM","EAsiaM","AusMCM"]
monsoon_names=["N. American Monsoon","Equatorial America","South American Monsoon","West African Monsoon","southern African","South and South East Asian Monsoon","East Asian Monsoon","Australian-Maritime Continent Monsoon"]
AR6_monsoons.data= np.where((landsea.lat < 0) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) < 16), 2, AR6_monsoons.data) # South America
AR6_monsoons.data= np.where((landsea.lat > 0) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) < 11), 1, AR6_monsoons.data) # Equatorial America
AR6_monsoons.data= np.where(regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) < 6, 0, AR6_monsoons.data) # North America
AR6_monsoons.data= np.where((landsea.lat > 0) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) > 19) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) < 24), 3, AR6_monsoons.data) # West Africa
AR6_monsoons.data= np.where((landsea.lat < 0) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) > 19) & (regionmask.defined_regions.ar6.land.mask(landsea.lon, landsea.lat) < 28), 4, AR6_monsoons.data) # southern Africa
AR6_monsoons.plot()
print(monsoon_abbrevs)
AR6_monsoons


In [ ]:
#Next we need to open monsoon.geojson (from https://github.com/IPCC-WG1/Atlas/blob/main/reference-regions/monsoons.geojson) and start applying that to the split between the Asian monsoons.


In [ ]:
# Write this out to file...
AR6_masks_1x1.to_netcdf("AR6_masks_1x1.nc")